In [19]:
# Weights and Biases related imports
import wandb
from wandb.keras import WandbMetricsLogger

In [20]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical

def load_cifar10_batch(file_path):
    with open(file_path, 'rb') as file:
        batch = pickle.load(file, encoding='bytes')
    return batch

def load_cifar10_data(folder_path):
    train_data = []
    train_labels = []

    for i in range(1, 6):
        batch_file = f"{folder_path}/data_batch_{i}"
        batch = load_cifar10_batch(batch_file)
        train_data.append(batch[b'data'])
        train_labels.extend(batch[b'labels'])

    test_batch_file = f"{folder_path}/test_batch"
    test_batch = load_cifar10_batch(test_batch_file)
    test_data = test_batch[b'data']
    test_labels = test_batch[b'labels']

    train_data = np.vstack(train_data)
    train_labels = np.array(train_labels)
    test_labels = np.array(test_labels)

    return train_data, train_labels, test_data, test_labels

def preprocess_data(train_data, train_labels, test_data, test_labels):
    train_data = train_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    test_data = test_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)

    train_labels_onehot = to_categorical(train_labels)
    test_labels_onehot = to_categorical(test_labels)

    return train_data, train_labels_onehot, test_data, test_labels_onehot

cifar10_folder = 'cifar-10-batches-py'

train_data, train_labels, test_data, test_labels = load_cifar10_data(cifar10_folder)

x_train, y_train, x_test, y_test = preprocess_data(
    train_data, train_labels, test_data, test_labels
)

print("Train Data Shape:", x_train.shape)
print("Train Labels Shape:", y_train.shape)
print("Test Data Shape:", x_test.shape)
print("Test Labels Shape:", y_test.shape)

Train Data Shape: (50000, 32, 32, 3)
Train Labels Shape: (50000, 10)
Test Data Shape: (10000, 32, 32, 3)
Test Labels Shape: (10000, 10)


In [21]:
# os.environ['WANDB_NOTEBOOK_NAME'] = 'RUN_1'
wandb.login()

True

In [22]:
sweep_config = {
    'method': 'grid'
    }

metric = {
    'name': 'val_loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'dropout': {
          'values': [True, False]
        },
    'batchnorm': {
          'values': [True, False]
        },
    'regularization': {
          'values': [True, False]
        },
    }

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'earlystopping_patience': {
        'value': 10},
    'epochs': {
        'value': 100},
    'learning_rate': {
        'value': 0.000063
        },
    'batch_size': {
          'value': 64
        },
    })

In [23]:
import pprint

pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'batch_size': {'value': 64},
                'batchnorm': {'values': [True, False]},
                'dropout': {'values': [True, False]},
                'earlystopping_patience': {'value': 10},
                'epochs': {'value': 100},
                'learning_rate': {'value': 6.3e-05},
                'regularization': {'values': [True, False]}}}


In [24]:
sweep_id = wandb.sweep(sweep_config, project="CIFAR-10_Classification")

Create sweep with ID: pavh7fhm
Sweep URL: https://wandb.ai/takim/CIFAR-10_Classification/sweeps/pavh7fhm


In [25]:
import tensorflow as tf

# Define the AlexNet architecture
def create_model(dropout, batchnorm, regularization):

    model = tf.keras.Sequential()

    if regularization:
        model.add(tf.keras.layers.Conv2D(filters=96, kernel_size=(11, 11), strides=(2, 2), activation='relu', input_shape=(32, 32, 3), kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    else:
        model.add(tf.keras.layers.Conv2D(filters=96, kernel_size=(11, 11), strides=(2, 2), activation='relu', input_shape=(32, 32, 3)))
    if batchnorm:
        model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(1, 1)))


    if regularization:
        model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    else:
        model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding="same"))
    if batchnorm:
        model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(1, 1)))


    if regularization:
        model.add(tf.keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.001)))
        model.add(tf.keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.001)))
        model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    else:
        model.add(tf.keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"))
        model.add(tf.keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"))
        model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"))
    if batchnorm:
        model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(1, 1)))


    model.add(tf.keras.layers.Flatten())


    model.add(tf.keras.layers.Dense(units=4096, activation='relu'))
    if dropout:
        model.add(tf.keras.layers.Dropout(0.5))


    model.add(tf.keras.layers.Dense(units=4096, activation='relu'))
    if dropout:
        model.add(tf.keras.layers.Dropout(0.5))


    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    return model

In [26]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def train(config = None):
    with wandb.init(config=config):

        config = wandb.config

        tf.keras.backend.clear_session()
        model = create_model(config["dropout"], config["batchnorm"], config["regularization"])
        model.compile(
            optimizer = Adam(learning_rate=config["learning_rate"]),
            loss = "categorical_crossentropy",
            metrics = ["accuracy", tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top@3_accuracy')]
        )

        early_stopping = EarlyStopping(monitor='val_loss',
                                    patience=config["earlystopping_patience"],
                                    restore_best_weights=True)

        history = model.fit(x_train, y_train,
                                    epochs=config["epochs"],
                                    batch_size=config["batch_size"],
                                    validation_split=0.1,
                                    callbacks=[
                                        WandbMetricsLogger(log_freq='epoch'),
                                        early_stopping
                                    ], verbose=1
                                    )
        
        test_stats = model.evaluate(x_test, y_test)
        wandb.log({"test_loss": test_stats[0]})
        wandb.log({"test_acc": test_stats[1]})

        val_loss_history = history.history['val_loss']
        val_acc_history = history.history['val_accuracy']

        best_epoch_num = -1 if (len(val_loss_history) == 100 or len(val_loss_history) <= 10) else (len(val_loss_history) - 11)

        wandb.log({"best_val_loss": val_loss_history[best_epoch_num]})
        wandb.log({"best_val_acc": val_acc_history[best_epoch_num]})

In [27]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 5rjnd6zc with config:
wandb: 	batch_size: 64
wandb: 	batchnorm: True
wandb: 	dropout: True
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	learning_rate: 6.3e-05
wandb: 	regularization: True
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 20s 24ms/step - loss: 3.1316 - accuracy: 0.3609 - top@3_accuracy: 0.6912 - val_loss: 2.5721 - val_accuracy: 0.4730 - val_top@3_accuracy: 0.8032
Epoch 2/100
704/704 [==============================] - 16s 23ms/step - loss: 2.5331 - accuracy: 0.4875 - top@3_accuracy: 0.8034 - val_loss: 2.4125 - val_accuracy: 0.5180 - val_top@3_accuracy: 0.8278
Epoch 3/100
704/704 [==============================] - 16s 23ms/step - loss: 2.3299 - accuracy: 0.5413 - top@3_accuracy: 0.8378 - val_loss: 2.4212 - val_accuracy: 0.4900 - val_top@3_accuracy: 0.8060
Epoch 4/100
704/704 [==============================] - 16s 23ms/step - loss: 2.1516 - accuracy: 0.5838 - top@3_accuracy: 0.8604 - val_loss: 2.2385 - val_accuracy: 0.5332 - val_top@3_accuracy: 0.8436
Epoch 5/100
704/704 [==============================] - 16s 23ms/step - loss: 1.9946 - accuracy: 0.6194 - top@3_accuracy: 0.8797 - val_loss: 2.0361 - val_accuracy: 0.5942 - val_top@3_accuracy: 0.8732
Epoch

best_val_acc,▁
best_val_loss,▁
epoch/accuracy,▁▃▃▄▄▅▅▅▆▆▆▆▇▇▇▇▇█████
epoch/epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▆▅▅▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁
epoch/top@3_accuracy,▁▄▄▅▅▆▆▆▇▇▇▇▇█████████
epoch/val_accuracy,▁▂▂▃▅▅▄▅▇▆▇▇▇▆▇▆▆▇▇█▇█
epoch/val_loss,█▇▇▆▄▄▄▃▂▂▁▁▁▂▂▂▂▁▂▂▂▁
epoch/val_top@3_accuracy,▁▃▁▄▆▄▃▅▇▅█▇▇▄▇▇▆█▇█▇█
test_acc,▁


wandb: Agent Starting Run: 3z9nwfum with config:
wandb: 	batch_size: 64
wandb: 	batchnorm: True
wandb: 	dropout: True
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	learning_rate: 6.3e-05
wandb: 	regularization: False
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 16s 23ms/step - loss: 2.0264 - accuracy: 0.3512 - top@3_accuracy: 0.6902 - val_loss: 1.6176 - val_accuracy: 0.4402 - val_top@3_accuracy: 0.7692
Epoch 2/100
704/704 [==============================] - 15s 22ms/step - loss: 1.4714 - accuracy: 0.4760 - top@3_accuracy: 0.7982 - val_loss: 1.4113 - val_accuracy: 0.5004 - val_top@3_accuracy: 0.8130
Epoch 3/100
704/704 [==============================] - 16s 22ms/step - loss: 1.3101 - accuracy: 0.5374 - top@3_accuracy: 0.8375 - val_loss: 1.2201 - val_accuracy: 0.5672 - val_top@3_accuracy: 0.8564
Epoch 4/100
704/704 [==============================] - 15s 22ms/step - loss: 1.1872 - accuracy: 0.5814 - top@3_accuracy: 0.8612 - val_loss: 1.4000 - val_accuracy: 0.4924 - val_top@3_accuracy: 0.8130
Epoch 5/100
704/704 [==============================] - 16s 22ms/step - loss: 1.0880 - accuracy: 0.6190 - top@3_accuracy: 0.8801 - val_loss: 1.2180 - val_accuracy: 0.5754 - val_top@3_accuracy: 0.8544
Epoch

...

best_val_acc,▁
best_val_loss,▁
epoch/accuracy,▁▂▃▄▄▅▅▆▆▆▇▇▇▇█████
epoch/epoch,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▆▅▄▄▄▃▃▂▂▂▂▁▁▁▁▁▁
epoch/top@3_accuracy,▁▃▄▅▅▆▆▇▇▇▇████████
epoch/val_accuracy,▁▃▄▅▄▆▇▇█▇███▇█████
epoch/val_loss,█▆▅▃▄▂▁▁▁▁▂▂▂▄▄▄▅▅▅
epoch/val_top@3_accuracy,▁▄▃▆▄▆▇██▇███▇█▇▇▇█
test_acc,▁


wandb: Agent Starting Run: gi4pzwz4 with config:
wandb: 	batch_size: 64
wandb: 	batchnorm: False
wandb: 	dropout: False
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	learning_rate: 6.3e-05
wandb: 	regularization: False
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 15s 21ms/step - loss: 1.8000 - accuracy: 0.4079 - top@3_accuracy: 0.7379 - val_loss: 1.4543 - val_accuracy: 0.4712 - val_top@3_accuracy: 0.8074
Epoch 2/100
704/704 [==============================] - 14s 20ms/step - loss: 1.3464 - accuracy: 0.5200 - top@3_accuracy: 0.8263 - val_loss: 1.3094 - val_accuracy: 0.5346 - val_top@3_accuracy: 0.8332
Epoch 3/100
704/704 [==============================] - 14s 20ms/step - loss: 1.2034 - accuracy: 0.5731 - top@3_accuracy: 0.8578 - val_loss: 1.2319 - val_accuracy: 0.5604 - val_top@3_accuracy: 0.8562
Epoch 4/100
704/704 [==============================] - 14s 20ms/step - loss: 1.1035 - accuracy: 0.6108 - top@3_accuracy: 0.8775 - val_loss: 1.1658 - val_accuracy: 0.5908 - val_top@3_accuracy: 0.8638
Epoch 5/100
704/704 [==============================] - 14s 20ms/step - loss: 0.9915 - accuracy: 0.6493 - top@3_accuracy: 0.8984 - val_loss: 1.1157 - val_accuracy: 0.6116 - val_top@3_accuracy: 0.8696
Epoch

best_val_acc,▁
best_val_loss,▁
epoch/accuracy,▁▂▃▄▄▅▅▆▆▆▇▇▇███
epoch/epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▅▄▄▄▃▃▂▂▂▂▁▁▁
epoch/top@3_accuracy,▁▃▄▅▅▆▆▇▇▇██████
epoch/val_accuracy,▁▃▄▅▆▇▇▇▇▆▇▇▇███
epoch/val_loss,▅▃▃▂▁▁▁▁▂▄▃▄▄▅▆█
epoch/val_top@3_accuracy,▁▃▅▆▆████▇▇█▇██▇
test_acc,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
